# Tratamento das base de dados das localizações dos radares eletrônicos

### Localização das bases

#### ANTT: https://dados.antt.gov.br/dataset/radar/resource/0e56291a-1092-45d5-b89a-d66b2a89dff4

#### DNIT: https://servicos.dnit.gov.br/multas/informacoes/equipamentos-fiscalizacao

In [12]:
%pip install nbformat
%pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 113.4/113.4 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 90.4/90.4 kB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import folium

In [3]:
df_antt = pd.read_csv("dados/loc_radar/antt.csv", encoding="latin1", sep=";")
df_antt.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
0,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"483,850",Betim,Principal,Crescente,Ativo,NaN,"-19,959486","-44,085386",80,60
1,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"493,290",Betim,Principal,Crescente,Ativo,NaN,"-19,964349","-44,171924",80,60
2,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"525,350",Brumadinho,Principal,Crescente,Ativo,NaN,"-20,146664","-44,323897",80,60
3,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"528,350",Brumadinho,Principal,Decrescente,Ativo,NaN,"-20,167194","-44,334434",80,60
4,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"568,890",Itaguara,Principal,Decrescente,Ativo,NaN,"-20,410679","-44,526628",80,60


In [4]:
df_antt.shape

(952, 15)

In [5]:
print(df_antt.columns)

Index(['concessionaria', 'ano_do_pnv_snv', 'tipo_de_radar', 'rodovia', 'uf',
       'km_m', 'municipio', 'tipo_pista', 'sentido', 'situacao',
       'data_da_inativacao', 'latitude', 'longitude', 'velocidade_leve',
       'velocidade_pesado'],
      dtype='object')


In [6]:
# Remove linhas sem coordenadas
df = df_antt.dropna(subset=['latitude', 'longitude'])

In [7]:
df_antt.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
0,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"483,850",Betim,Principal,Crescente,Ativo,NaN,"-19,959486","-44,085386",80,60
1,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"493,290",Betim,Principal,Crescente,Ativo,NaN,"-19,964349","-44,171924",80,60
2,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"525,350",Brumadinho,Principal,Crescente,Ativo,NaN,"-20,146664","-44,323897",80,60
3,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"528,350",Brumadinho,Principal,Decrescente,Ativo,NaN,"-20,167194","-44,334434",80,60
4,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"568,890",Itaguara,Principal,Decrescente,Ativo,NaN,"-20,410679","-44,526628",80,60


In [8]:
df_antt['latitude'] = df_antt['latitude'].str.replace(',', '.')
df_antt['longitude'] = df_antt['longitude'].str.replace(',', '.')

df_antt['latitude'] = pd.to_numeric(df_antt['latitude'], errors='coerce')
df_antt['longitude'] = pd.to_numeric(df_antt['longitude'], errors='coerce')


In [9]:
df_antt.head()

,concessionaria,ano_do_pnv_snv,tipo_de_radar,rodovia,uf,km_m,municipio,tipo_pista,sentido,situacao,data_da_inativacao,latitude,longitude,velocidade_leve,velocidade_pesado
0,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"483,850",Betim,Principal,Crescente,Ativo,NaN,-19.959486,-44.085386,80,60
1,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"493,290",Betim,Principal,Crescente,Ativo,NaN,-19.964349,-44.171924,80,60
2,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"525,350",Brumadinho,Principal,Crescente,Ativo,NaN,-20.146664,-44.323897,80,60
3,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"528,350",Brumadinho,Principal,Decrescente,Ativo,NaN,-20.167194,-44.334434,80,60
4,AUTOPISTA FERNÃO DIAS,2007,Controlador,BR-381,MG,"568,890",Itaguara,Principal,Decrescente,Ativo,NaN,-20.410679,-44.526628,80,60


In [15]:
lat_centro = df_antt['latitude'].mean()
lon_centro = df_antt['longitude'].mean()

# Cria mapa com zoom inicial
mapa = folium.Map(location=[lat_centro, lon_centro], zoom_start=5)

# Adiciona marcadores
for _, row in df_antt.iterrows():
    tooltip = f"""
    <b>Concessionária:</b> {row['concessionaria']}<br>
    <b>Rodovia:</b> {row['rodovia']}<br>
    <b>Município:</b> {row['municipio']}<br>
    <b>Tipo:</b> {row['tipo_de_radar']}<br>
    <b>Vel. leve:</b> {row['velocidade_leve']} km/h
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=tooltip,
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(mapa)

# Salva e/ou exibe o mapa
mapa.save('radares_antt.html')